In [1]:
# !pip install smolagents -U 
# !pip install --upgrade groq
# !pip install --upgrade smolagents
# !pip install --upgrade torch
# !pip install --upgrade scipy
# !pip install litellm

## HFApiModel Serverless API

In [2]:
from huggingface_hub import login

login()

In [3]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()], 
    model=HfApiModel()
)

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = web_search(query="best music recommendations for a party at Wayne's mansion")                   
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
The best party songs 1. "September" - Earth, Wind & Fire (1978) Quite possibly the best party song. An infectious 
mix of funk and soul, "September" is celebrated for its upbeat melody and "ba-dee-ya" chorus, making it a timeless 
dance favorite.

[late nights in the wayne manor part 2 | a bruce wayne rock/blues ...](https://m.youtube.com/watch?v=DBRSHk2XpmU)
these are songs that bruce wayne listens to while wandering the halls of the great wayne manor. this is part 2.find
this playlist on my spotify: https://open...

[Best Songs To Party: DJ's Ultimate Party Songs Playlist - 
Top40Weekly.com](https://top40weekly.com/best-songs-to-party/)
If you're looking for the ultimate playlist to get your party started, look no further than this best party songs 
playlist. Whether you're hosting a birthday bash, a backyard barbecue, or just a night in with friends, the right 
music can make all the difference. From classic hits to modern bangers, we've compiled a list of the best songs ...

[50 Songs on Every Event Planner's Playlist - 
Eventbrite](https://www.eventbrite.com/blog/event-planning-playlist-ds00/)
Music sets the mood and provides the soundtrack (literally) for a memorable and exciting time. While the right 
songs can enhance the experience, the wrong event music can throw off the vibe. For example, fast-paced songs 
probably aren't the best fit for a formal gala. And smooth jazz is likely to lull your guests at a motivational 
conference.

[Similar Songs Finder | Spotify Playlist Generator - Chosic](https://www.chosic.com/playlist-generator/)
Need to find similar songs to a song you like? To music you love? A playlist created for you in seconds! With 
Spotify Playlist Generator online tool, create awesome playlists in seconds, featuring similar songs to those you 
love. Whether by songs, artists, genres, moods, or based on a playlist, just start with what you have in mind, and 
it will provide you with plenty of song recommendations ...

[40 Best Party Songs | Songs To Dance To, Ranked By Our Editors - Time 
Out](https://www.timeout.com/music/best-party-songs)
The 40 best party songs ever made. It's party time guys - don your dancing shoes and have your speaker at the 
ready. Thursday 13 February 2025. Share. Copy Link. Facebook Twitter Pinterest ...

[200 Classic House Party Songs Everyone Knows | The Best Popular Party 
Songs](https://www.ispytunes.com/post/house-party-songs)
Next, let's get into reviewing 20 of the best songs for your next house party. 1. rockstar by Post Malone (feat. 21
Savage) "rockstar" by Post Malone (feat. 21 Savage) is an incredibly popular song with a great instrumental and an 
infectious chorus, which makes it a perfect song for playing at parties. The whole song is like a giant energy ...

[Lil Wayne's A Milli: Ultimate Party Audio Experience - 
TikTok](https://www.tiktok.com/@jbl310spkr/video/7489588657968483606)
Keywords: Lil Wayne A Milli song, best speaker for parties, JBL PartyBox audio experience, enhance audio for 
gatherings, Fetty Wap music party, high-quality portable speakers, party audio technology, Bluetooth speaker for 
events, A Milli party anthem, party music recommendations

[Top Playlists; Find Your Perfect Music Mix | 
iHeart](https://www.iheart.com/playlist/collections/perfect-for/activities/party/)
Discover and stream the best playlists, handcrafted by the music experts at iHeart. Discover and stream the best 
playlists, handcrafted by the music experts at iHeart. For You; Your Library; Live Radio ... Hip Hop Tailgate 
Party. Rap Party. Rock Party. Country Tailgate. Girls Night Out. 2010s Party Hits. Al Ritmo De Reggaeton. Sunday 
Brunch ...

[The Best Songs With Mansion in the Title - 
Ranker](https://www.ranker.com/list/the-best-songs-with-mansion-in-the-title/ranker-music)
This list ranks the best songs with mansion in

[Step 1: Duration 2.15 seconds| Input tokens: 2,085 | Output tokens: 86]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "recommended_playlist": party_playlist,                                                                      
      "spotify_playlist_url": spotify_playlist_url                                                                 
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'recommended_playlist': ['September - Earth, Wind & Fire', 'Bill with Wings - Queen', 'Sharp 
Dressed Man - ZZ Top', "Sweet Child o' Mine - Guns N' Roses", 'I Will Survive - Gloria Gaynor', 'Stay with Me - Sam
Smith', 'Bad Boys - Backstreet Boys', 'Maniac - Aerosmith', 'Sweet Dreams (Are Made of This) - Eurythmics', "Livin'
On a Prayer - Bon Jovi"], 'spotify_playlist_url': 'https://open.spotify.com/playlist/3fnQ0NF8gS15O6i93irYlM'}

[Step 4: Duration 5.87 seconds| Input tokens: 14,103 | Output tokens: 676]

{'recommended_playlist': ['September - Earth, Wind & Fire',
  'Bill with Wings - Queen',
  'Sharp Dressed Man - ZZ Top',
  "Sweet Child o' Mine - Guns N' Roses",
  'I Will Survive - Gloria Gaynor',
  'Stay with Me - Sam Smith',
  'Bad Boys - Backstreet Boys',
  'Maniac - Aerosmith',
  'Sweet Dreams (Are Made of This) - Eurythmics',
  "Livin' On a Prayer - Bon Jovi"],
 'spotify_playlist_url': 'https://open.spotify.com/playlist/3fnQ0NF8gS15O6i93irYlM'}

## Custom Agent

In [4]:
from smolagents import CodeAgent, tool, HfApiModel

@tool
def suggest_menu(occasion: str)-> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """

    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

In [4]:
Agent= CodeAgent(tools= [suggest_menu], model= HfApiModel())
Agent.run("Generate a menu for the party")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Generate a menu for the party                                                                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  menu = suggest_menu(occasion="casual")                                                                           
  final_answer(menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Pizza, snacks, and drinks.

[Step 1: Duration 5.62 seconds| Input tokens: 2,105 | Output tokens: 70]

'Pizza, snacks, and drinks.'

## Agent with GROQ

### Code execution has strict security measures - imports outside a predefined safe list are blocked by default.

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
os.environ["GROK_TOKEN"]=os.getenv("GROK_TOKEN")

In [42]:
from smolagents import CodeAgent
from smolagents.models import LiteLLMModel
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROK_TOKEN")

model = LiteLLMModel(
    model_id="groq/llama3-70b-8192",
    model="groq/llama3-8b-8192",  
    api_base="https://api.groq.com/openai/v1",
    max_tokens=500  # Removed 'system_prompt'
)

# Required for Groq compatibility
model._flatten_messages_as_text = True

from smolagents import CodeAgent
import datetime

agent = CodeAgent(tools=[], model=model, add_base_tools=True, additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     4. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ LiteLLMModel - groq/llama3-70b-8192 ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  current_time = datetime.datetime.now()                                                                           
  print("Current time:", current_time)                                                                             
                                                                                                                   
  task_times = [30, 60, 45, 45]                                                                                    
  total_time = sum(task_times)                                                                                     
  print("Total time to prepare the party:", total_time, "minutes")                                                 
                                                                                                                   
  party_ready_time = current_time + datetime.timedelta(minutes=total_time)                                         
  print("The party will be ready at:", party_ready_time)                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Current time: 2025-03-30 15:17:26.754737
Total time to prepare the party: 180 minutes
The party will be ready at: 2025-03-30 18:17:26.754737

Out: None

[Step 1: Duration 1.44 seconds| Input tokens: 2,176 | Output tokens: 150]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  party_ready_time_str = party_ready_time.strftime("%Y-%m-%d %H:%M")                                               
  final_answer("The party will be ready at " + party_ready_time_str)                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The party will be ready at 2025-03-30 18:17

[Step 2: Duration 1.27 seconds| Input tokens: 4,711 | Output tokens: 224]

'The party will be ready at 2025-03-30 18:17'

## Integrating Tools

In [6]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, Tool, tool, VisitWebpageTool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.
    
    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }
    
    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)
    
    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""
    
    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }
    
    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }
        
        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


In [7]:

# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(), 
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
    ], 
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2,
)

agent.run("Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme    │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To generate the best playlist for a 'villain masquerade' party at Wayne's mansion, I will first use the   
`superhero_party_theme_generator` tool to get a creative theme idea and then use web search to find a playlist that
matches this theme.                                                                                                
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
theme_idea = superhero_party_theme_generator(category="villain masquerade")                                        
print(f"The villain masquerade theme idea is: {theme_idea}")                                                       
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  theme_idea = superhero_party_theme_generator(category="villain masquerade")                                      
  print(f"The villain masquerade theme idea is: {theme_idea}")                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The villain masquerade theme idea is: Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic 
Batman villains.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The visited Spotify playlist "Arkham Asylum (Batman, Joker, Gotham, Harley Quinn, Scarecrow, villains,    
rogues)" contains a mix of songs that fit the theme of a villain masquerade party at Wayne's mansion. From the list
of songs, we can compile a final playlist.                                                                         
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_playlist = [                                                                                                 
    "PANIC! - Smith & Myers",                                                                                      
    "Smile - Wolf Alice",                                                                                          
    "Violence and Spiders - SAINT PHNX",                                                                           
    "Mad Hatter - Melanie Martinez",                                                                               
    "Pretty Pills for Broken Hearts - Cloudy June",                                                                
    "Bad Guy - The Cog is Dead",                                                                                   
    "Under My Skin - Jukebox The Ghost",                                                                           
    "Doctor - Truslow",                                                                                            
    "Good Time (1:08) - Jagwar Twin",                                                                              
    "Pink (Freak) - Elliot Lee",                                                                                   
    "Lunatics and Slaves - Sin Shake Sin",                                                                         
    "Cut the Cord - Shinedown",                                                                                    
    "No Place Like Home - Todrick Hall",                                                                           
    "Imaginary Parties - Superfruit",                                                                              
    "Let 'Em Talk - Rosie and the Riveters",                                                                       
    "Sun Goes Down - New Medicine",                                                                                
    "In My Head - Mike Shinoda, Kailee[38;2;230;237;2

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_playlist = [                                                                                               
      "PANIC! - Smith & Myers",                                                                                    
      "Smile - Wolf Alice",                                                                                        
      "Violence and Spiders - SAINT PHNX",                                                                         
      "Mad Hatter - Melanie Martinez",                                                                             
      "Pretty Pills for Broken Hearts - Cloudy June",                                                              
      "Bad Guy - The Cog is Dead",                                                                                 
      "Under My Skin - Jukebox The Ghost",                                                                         
      "Doctor - Truslow",                                                                                          
      "Good Time (1:08) - Jagwar Twin",                                                                            
      "Pink (Freak) - Elliot Lee",                                                                                 
      "Lunatics and Slaves - Sin Shake Sin",                                                                       
      "Cut the Cord - Shinedown",                                                                                  
      "No Place Like Home - Todrick Hall",                                                                         
      "Imaginary Parties - Superfruit",                                                                            
      "Let 'Em Talk - Rosie and the Riveters",                                                                     
      "Sun Goes Down - New Medicine",                                                                              
      "In My Head - Mike Shinoda, Kailee Morgue",                                                                  
      "Black Sheep - Metric",                                                                                      
      "The Takeover - New Medicine",                                                                               
      "Black Sheep - Metric",                                                                                      
      "I Don't Take Insults Lightly - Madds Buckley",                                                              
      "Someone's Gonna Die Tonight - Gin Wigmore",                                                                 
      "Filthy Freaks - BONES UK",                                                                                  
      "The Hand That Feeds - Nine Inch Nails",                                                                     
      "Look Who's Inside Again - Bo Burnham",                                                                      
      "Dangerous - New Medicine",                                                                                  
      "Can't Go to Hell - Sin Shake Sin",                                                                          
      "Congratulations, You're in a Cult! - Sin Shake Sin"                                                         
  ]                                                                                                                
  final_answer(final_playlist)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['PANIC! - Smith & Myers', 'Smile - Wolf Alice', 'Violence and Spiders - SAINT PHNX', 'Mad 
Hatter - Melanie Martinez', 'Pretty Pills for Broken Hearts - Cloudy June', 'Bad Guy - The Cog is Dead', 'Under My 
Skin - Jukebox The Ghost', 'Doctor - Truslow', 'Good Time (1:08) - Jagwar Twin', 'Pink (Freak) - Elliot Lee', 
'Lunatics and Slaves - Sin Shake Sin', 'Cut the Cord - Shinedown', 'No Place Like Home - Todrick Hall', 'Imaginary 
Parties - Superfruit', "Let 'Em Talk - Rosie and the Riveters", 'Sun Goes Down - New Medicine', 'In My Head - Mike 
Shinoda, Kailee Morgue', 'Black Sheep - Metric', 'The Takeover - New Medicine', 'Black Sheep - Metric', "I Don't 
Take Insults Lightly - Madds Buckley", "Someone's Gonna Die Tonight - Gin Wigmore", 'Filthy Freaks - BONES UK', 
'The Hand That Feeds - Nine Inch Nails', "Look Who's Inside Again - Bo Burnham", 'Dangerous - New Medicine', "Can't
Go to Hell - Sin Shake Sin", "Congratulations, You're in a Cult! - Sin Shake Sin"]

[Step 4: Duration 28.47 seconds| Input tokens: 16,229 | Output tokens: 699]

['PANIC! - Smith & Myers',
 'Smile - Wolf Alice',
 'Violence and Spiders - SAINT PHNX',
 'Mad Hatter - Melanie Martinez',
 'Pretty Pills for Broken Hearts - Cloudy June',
 'Bad Guy - The Cog is Dead',
 'Under My Skin - Jukebox The Ghost',
 'Doctor - Truslow',
 'Good Time (1:08) - Jagwar Twin',
 'Pink (Freak) - Elliot Lee',
 'Lunatics and Slaves - Sin Shake Sin',
 'Cut the Cord - Shinedown',
 'No Place Like Home - Todrick Hall',
 'Imaginary Parties - Superfruit',
 "Let 'Em Talk - Rosie and the Riveters",
 'Sun Goes Down - New Medicine',
 'In My Head - Mike Shinoda, Kailee Morgue',
 'Black Sheep - Metric',
 'The Takeover - New Medicine',
 'Black Sheep - Metric',
 "I Don't Take Insults Lightly - Madds Buckley",
 "Someone's Gonna Die Tonight - Gin Wigmore",
 'Filthy Freaks - BONES UK',
 'The Hand That Feeds - Nine Inch Nails',
 "Look Who's Inside Again - Bo Burnham",
 'Dangerous - New Medicine',
 "Can't Go to Hell - Sin Shake Sin",
 "Congratulations, You're in a Cult! - Sin Shake Sin"]

### Installing some dependencies

In [9]:
# pip install opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

In [11]:
import os
import base64

LANGFUSE_PUBLIC_KEY=os.getenv("LANGFUSE_PRIV")
LANGFUSE_SECRET_KEY=os.getenv("LANGFUSE_PUB")

LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [12]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [19]:
from smolagents import CodeAgent
from smolagents.models import LiteLLMModel
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROK_TOKEN")

model = LiteLLMModel(
    model_id="groq/llama3-70b-8192",
    model="groq/llama3-8b-8192",  
    api_base="https://api.groq.com/openai/v1",
    max_tokens=500  # Removed 'system_prompt'
)

# Required for Groq compatibility
model._flatten_messages_as_text = True

In [20]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(tools=[], model=model)
alfred_agent = agent.from_hub('sergiopaniego/AlfredAgent', trust_remote_code=True)
alfred_agent.run("give me some all timer rock songs")